**Decision Tree From Scratch**:


First I import the needed libraries.I develop the values in pandas since it is easier to work with.

In [58]:
import numpy as np
import pandas as pd
import math

In [59]:
data = {'Alt':['yes','yes','no','yes','yes','no','no','no','no','yes','no','yes'],
        'Bar':['no','no','yes','no','no','yes','yes','no','yes','yes','no','yes'],
        'Fri':['no','no','no','yes','yes','no','no','no','yes','yes','no','yes'],
        'Hun':['yes','yes','no','yes','no','yes','no','yes','no','yes','no','yes'],
        'Pat':['some','full','some','full','full','some','none','some','full','full','none','full'],
        'Price':[3,1,1,1,3,2,1,2,1,3,1,1],
        'Rain':['no','no','no','yes','no','yes','yes','yes','yes','no','no','no'],
        'Res':['yes','no','no','no','yes','yes','no','yes','no','yes','no','no'],
        'Type':['french','thai','burger','thai','french','italian','burger','thai','burger','italian','thai','burger'],
        'Est':[1,3,1,2,4,1,1,1,4,2,1,3],
        'y':['yes','no','yes','yes','no','yes','no','yes','no','no','no','yes']
        }


df = pd.DataFrame(data,columns=['Alt','Bar','Fri','Hun','Pat','Price','Rain','Res','Type','Est','y'])

Now, for making the tree, I use the **mutual information** formula.


Since H(y) is the same in entropies for all of the features, I only calculate the **conditional entropy**: H(y|x) and by minizing it, I find the nodes at each level of the tree.


The below code is a function that calculates H(y|x) for each feature.

In [60]:
def conditional_entropy(df,feature):
    eps = np.finfo(float).eps
    labels = df.keys()[-1]   
    label_values = df[labels].unique() 
    feature_values = df[feature].unique()
    entropy2 = 0
    for variable in feature_values:
        entropy = 0
        for label in label_values:
            num = len(df[feature][df[feature] == variable][df[labels] == label])
            den = len(df[feature][df[feature] == variable])
            probability = num/(den+eps)
            entropy += -probability * math.log2(probability+eps)
            fraction2 = den/len(df)
            entropy2 += -fraction2*entropy
    return abs(entropy2)

I define a function that finds the next node with minimum H(y|x):

In [61]:
def next_level (df):
    features_entropies = []
    for key in df.keys()[:-1]:
        features_entropies.append(conditional_entropy(df,key))
    return df.keys()[:-1][np.argmin(features_entropies)]

Now I define the final functin which implements the entire tree by using the above functions:

In [62]:
def get_subtable(df, node,value):
  return df[df[node] == value].reset_index(drop=True)
  
def buildTree(df,tree=None): 
    feature = next_level(df)
    feature_Values = np.unique(df[feature])

    if tree is None:                    
        tree={}
        tree[feature] = {}
  
    for value in feature_Values:
        subtable = get_subtable(df,feature,value)
        Value,counts = np.unique(subtable['y'],return_counts=True)                        
        
        if len(counts)==1:
            tree[feature][value] = Value[0]                                                    
        else:        
            tree[feature][value] = buildTree(subtable)
                   
    return tree

I print the implemented tree as below:

In [63]:
import pprint
tree = buildTree(df)
pprint.pprint(tree)

{'Pat': {'full': {'Hun': {'no': 'no',
                          'yes': {'Type': {'burger': 'yes',
                                           'italian': 'no',
                                           'thai': {'Fri': {'no': 'no',
                                                            'yes': 'yes'}}}}}},
         'none': 'no',
         'some': 'yes'}}


As it can be seen from the above tree, by calculating the entropy of each feature at each level, the first node is the 'pat' feature, then it is 'hun', then 'type' and finally 'fri'.

In [ ]:
entropy_y = 0
y_values = df.y.unique()
for i in y_values:
    probability = df.y.value_counts()[i]/len(df.y)  
    entropy_y += -probability * math.log2(probability)